In [ ]:
import numpy as np
from lookup_table import CaseNum2EdgeOffset, getCaseNum
import trimesh
import os
import time

In [ ]:
def marching_cube(thres,cells):
    # vertices use dictionary to avoid duplicate axes
    vertex_array = {}
    face_array = []
    t1 = time.time()
    # -------------------TODO------------------ 
    # compute vertices and faces
    # vertices: [N, 3]
    # faces: [M, 3], e.g. np.array([[0,1,2]]) means a triangle composed of vertices[0], vertices[1] and vertices[2]
    # for-loop is allowed to reduce difficulty
    for i in range(cells.shape[0] - 1):
        for j in range(cells.shape[1] - 1):
            for k in range(cells.shape[2] - 1):
                case_nums = getCaseNum(i, j, k, thres, cells)
                for l in range(0, len(case_nums) - 1, 3):
                    if case_nums[l] == -1:
                        break
                    triangle = case_nums[l:l+3]
                    
                    def f(edge):
                        edge = CaseNum2EdgeOffset[edge]
                        p0 = np.array(edge[:3])
                        p1 = np.array(edge[3:])
                        v0 = cells[i + p0[0], j + p0[1], k + p0[2]]
                        v1 = cells[i + p1[0], j + p1[1], k + p1[2]]
                        p = np.array([i, j, k]) + p0 + (thres - v0) / (v1 - v0) * (p1 - p0)
                        return tuple(np.round(p, 6))
                    
                    vertices = list(map(f, triangle))
                    for v in vertices:
                        vertex_array.setdefault(v, len(vertex_array))
                    face_array.append([vertex_array[v] for v in vertices])
    # -------------------TODO------------------ 
    t2 = time.time()
    print("\nTime taken by algorithm\n"+'-'*40+"\n{} s".format(t2-t1))
    vertex_array = list(vertex_array.keys())
    return np.array(vertex_array), np.array(face_array)

In [ ]:
# reconstruct these two animals
shape_name_lst = ['spot', 'bob']
for shape_name in shape_name_lst:
    data = np.load(os.path.join('data', shape_name + '_cell.npy'))
    verts, faces = marching_cube(0, data)
    print(verts.shape, faces.shape)
    mesh = trimesh.Trimesh(vertices=verts, faces=faces)
    mesh_txt = trimesh.exchange.obj.export_obj(mesh)
    with open(os.path.join('../results', shape_name + '.obj'),"w") as fp:
        fp.write(mesh_txt)